In [2]:
import pandas as pd
import numpy as np
import numbers
import operator
import re
from __future__ import division

In [3]:
ID_var_name = 'pet_mr_radiomics'
n_trial = 1000
k_fold = 3

rootdir = '/Users/shuang/Documents/Proj_Radiomics/Data/her2'
the_mri_tp = 2
the_mri_bin_width = 5
the_pet_bin_width = 0.1
im_dir = '{}/her2_Analysis/PETMRI/PETbinwidth{:.1f}_MRItp{}_binwidth{}'.format(rootdir,the_pet_bin_width, the_mri_tp, the_mri_bin_width)

# the selected model to use for all DF outcome
# the_clf_name = 'ElasticNet'
# df_yr = range(1,6,1)
# outcome_names = ['DF_{}yr'.format(yy) for yy in df_yr]

the_clf_name = 'L2lbfgsLogReg'
outcome_names = ['Tumor_Grade_Binary']

In [ ]:
# find 'important' features for ElasticNet algorithm
for oc in outcome_names:
    json_fname = '{}/Learner/{}_IDV{}_DV{}_Trial{}_{}folds.json'.format(im_dir, the_clf_name, ID_var_name, oc, n_trial, k_fold)
    df_learning_output = pd.read_json(json_fname)
    for ii in df_learning_output.index:
        feat_imp = np.array(df_learning_output['feat_importance'][ii])
        feat_names = np.array(df_learning_output['feat_name'][ii])
        
        # find a list of features that are not dropped by ElasticNet
        lst_tmp = feat_names[feat_imp != 0.0]
        if ii == 0:
            #initialize the dictionary
            dct_feat_tally = {}
            for ss in lst_tmp:
                dct_feat_tally[ss] = 1
        else:
            for ss in lst_tmp:
                if ss in dct_feat_tally.keys():
                    dct_feat_tally[ss] += 1
                else:
                    dct_feat_tally[ss] = 1
    
    # sort the feat tally dictionary
    sorted_dct_feat_tally = sorted(dct_feat_tally.items(), key=operator.itemgetter(1), reverse=True)
    top10_feat = sorted_dct_feat_tally[0:10]
    
    max_count = len(df_learning_output)
    print('outcome: {}\n'.format(oc))
    for (feat, count) in top10_feat:
        print('{} ({})'.format(feat, float(count/max_count)))
    print('\n')
#     print(sorted_dct_feat_tally)
#     for k, value in sorted_dct_feat_tally.items():
#         print('feat name: {}, count: {}'.format(k, value))
    
    

In [6]:
# find 'important' features for Logistic Regression\
coeff_thresh = 0.01
for oc in outcome_names:
    json_fname = '{}/Learner/{}_IDV{}_DV{}_Trial{}_{}folds.json'.format(im_dir, the_clf_name, ID_var_name, oc, n_trial, k_fold)
    df_learning_output = pd.read_json(json_fname)
#     print(len(df_learning_output.index.tolist()))
    for ii in df_learning_output.index:
        feat_imp = np.array(df_learning_output['feat_importance'][ii])
        feat_names = np.array(df_learning_output['feat_name'][ii])

        # find a list of features that are not dropped by ElasticNet
        lst_tmp = feat_names[np.abs(feat_imp) > coeff_thresh]
        if ii == 0:
            #initialize the dictionary
            dct_feat_tally = {}
            for ss in lst_tmp:
                dct_feat_tally[ss] = 1
        else:
            for ss in lst_tmp:
                if ss in dct_feat_tally.keys():
                    dct_feat_tally[ss] += 1
                else:
                    dct_feat_tally[ss] = 1
    
    # sort the feat tally dictionary
    sorted_dct_feat_tally = sorted(dct_feat_tally.items(), key=operator.itemgetter(1), reverse=True)
    top10_feat = sorted_dct_feat_tally[0:10]
    
    max_count = len(df_learning_output)
    print('outcome: {}\n'.format(oc))
    for (feat, count) in top10_feat:
        print('{} ({:.1%})'.format(feat, float(count/max_count)))
    print('\n')

(array([-0.0258225 , -0.01518348,  0.03563678, -0.02458551,  0.00206473,
        0.00687059, -0.02282641, -0.00646505, -0.00464322, -0.03342172,
       -0.02866758,  0.01878352, -0.03250961, -0.0172686 , -0.02874081,
       -0.02373984, -0.02907081, -0.02892077, -0.02306865, -0.0231096 ,
        0.00289675, -0.0156292 , -0.02249804,  0.01713002, -0.01112963,
        0.01413147,  0.012689  ,  0.00532462,  0.01590337,  0.00794244,
        0.0187878 , -0.02525588, -0.01503631, -0.02371352,  0.00782661,
       -0.01133406, -0.02249804, -0.02332878, -0.02525588, -0.02317415,
       -0.02199317, -0.02675036,  0.01043398,  0.03996115, -0.0144827 ,
        0.03047159, -0.01886497,  0.02423152, -0.00213131, -0.03293564,
        0.03032448,  0.01326739,  0.04698611, -0.0130042 , -0.0317089 ,
        0.04123355,  0.00414635, -0.01809286,  0.01427789,  0.02982991,
        0.02278305,  0.01158311,  0.03025274,  0.02784131,  0.0127952 ,
        0.03930428,  0.03829733, -0.02842949,  0.03978501, -0.0

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, auc, roc_curve
from scipy.stats import zscore

In [ ]:
# read in all radiomic data
fname = '{}/data_all.csv'.format(im_dir)
df_data = pd.read_csv(fname)
pat = re.compile('texture_|FOstats_|ShapeSize_')
feat_names = [ss for ss in df_data.columns.tolist() if pat.match(ss)]
feat_tag = 'pet_mr_radiomics'

# scale the features to z-score
df_data[feat_names] = df_data[feat_names].apply(zscore)

In [ ]:
len(df_data.MRN.unique())

In [ ]:
#read the learning output data and more
fname = '{}/Learner/CLF_output_all.csv'.format(im_dir)
df_clf_all = pd.read_csv(fname)

# simplify clf name
def simplify_clf_name_func(x):
    if re.search('(.+)LogReg',x):
        return 'LogReg'
    else:
        return x
df_clf_all['clf_name_simplify'] = df_clf_all['clf_name'].map(simplify_clf_name_func)
# print(df_clf_all.ix[:,['clf_name_simplify', 'clf_name']])

# # drop the one with Random Forest since we would like to use logistic regression only
# df_clf_all.drop(df_clf_all[(df_clf_all.Dep_var == 'DF_1yr') & (df_clf_all.clf_name == 'RandomForest')].index, inplace=True)
# idx = df_clf_all.groupby('Dep_var').apply(lambda df: df.AUC_mean.argmax())  

dct_clf = {'ElasticNet': SGDClassifier(), 'LogReg': LogisticRegression(), 'RandomForest': RandomForestClassifier(),
          'SVM': SVC()}


In [ ]:
test = [idx[3]]
print(test)
for ii in test:
#     clf_name, clf_name_simple, oc, n_trial, k_fold = df_clf_all.ix[ii, ['clf_name', 'clf_name_simplify', 'Dep_var', 'n_trial', 'k_fold']].tolist()
    json_fname = '{}/Learner/{}_IDV{}_DV{}_Trial{}_{}folds.json'.format(im_dir, the_clf_name, feat_name, oc, n_trial, k_fold)
    df_learning_output = pd.read_json(json_fname)
    lst_data_all = []
    for iid in df_learning_output.index:
        dct_param = df_learning_output['best_params'][iid]
        if iid == 0: # determine what to average on the first round
            lst_num_var = []
            for k, val in dct_param.items():
               if isinstance(val, numbers.Number):
                lst_num_var.append(k)
            lst_str_var = set(lst_num_var).symmetric_difference(dct_param.keys())
        lst_tmp = [dct_param[s] for s in lst_num_var]
        tmp = dict(zip(lst_num_var, lst_tmp))
        lst_data_all.append(tmp)
        
    df_data_all = pd.DataFrame(lst_data_all)
    df_data_all = df_data_all.ix[:, lst_num_var]
    
    # create the final parameter with the average over all numerical variables
    final_param = dict(df_data_all.mean())
    
    # add the string parameter variabls
    for ss in lst_str_var:
        final_param[ss] = dct_param[ss]
    print(final_param)
        
    # get all the needed radiomic data
    df_tmp = df_data.ix[:,feat_names + [oc]]
    df_tmp = df_tmp.dropna()
    X, y = df_tmp.ix[:,feat_names].as_matrix(), df_tmp.ix[:,oc].astype('int').as_matrix()
    clf = dct_clf[the_clf_name]
    print(clf)
    clf.set_params(**final_param)
    clf.fit(X, y)
    feat_importance = getattr(clf, 'feature_importances_', None)
    if feat_importance is None and hasattr(clf, 'coef_'):
        feat_importance = clf.coef_
    print(feat_importance)


In [ ]:
feat_imp = np.array(df_learning_output['feat_importance'][4])
feat_name = np.array(df_learning_output['feat_name'][4])
print(feat_name[feat_imp != 0.0])
print(feat_imp)

In [ ]:
# for easier interpretbaility of output, we will use elasticNet for reporting feature importance for all outcome!